Standard imports.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

Propagating wavefront. It serves as reference and as initial condition.

In [ ]:
def propagatingFunction(x):
    return np.exp(-np.power(x, 2) / (2. * np.power(np.pi / 4., 2)))

Solution function. Rows span time, columns span space. We have ```M + 1``` rows: ```M``` iterations + ```1``` initial condition. We have ```N + 1``` columns: ```N``` discretization points + ```1``` boundary condition.

Leapfrog.

In [ ]:
def leapfrog(x, t, v, alpha):

  u = np.zeros(((M + 1), N + 1))

  # --- Initial condition
  u[0]      = propagatingFunction(x - v * t[0]) 
  
  # --- First step
  u[1, 1 : N]       = u[0, 1 : N] - 0.5 * alpha * (u[0, 2 : N + 1] - u[0, 0 : N - 1])
  u[1, 0]           = propagatingFunction(x[0] - v * t[1]) 

  Q = (1. - alpha) / (1. + alpha)
  for l in range(1, M):
    u[l + 1, 1 : N]     = u[l - 1, 1 : N] - alpha * (u[l, 2 : N + 1] - u[l, 0 : N - 1]) 
    # --- Boundary condition
    u[l + 1, 0]         = propagatingFunction(x[0] - v * t[l + 1])    
    u[l + 1, N]         = u[l, N - 1] - Q * u[l + 1, N - 1] + Q * u[l, N] 

  return u

Parameters.

In [ ]:
xmin  = 0                                    # --- Left boundary of the simulation domain
xmax  = 2. * np.pi                           # --- Right boundary of the simulation domain
t_0   = 0.                                   # --- Initial time
t_f   = 15.                                  # --- Final time
M     = 200                                  # --- Number of time steps
N     = 165                                  # --- Number of space mesh points
v     = 0.5                                  # --- Wave speed

Space-time discretization.

In [ ]:
x, dx = np.linspace(xmin, xmax, N + 1, retstep = True)

t, dt = np.linspace(t_0, t_f, M + 1, retstep = True)
            
alpha = v * dt / dx

Define reference solution.

In [ ]:
X, T = np.meshgrid(x, t)           

uRef = propagatingFunction(X - v * T)

Compute solution.

In [ ]:
u = leapfrog(x, t, v, alpha)

Animate.

In [ ]:
fig, ax = plt.subplots()
plt.close()

ax.set_xlim(( xmin, xmax))
ax.set_ylim((    0, 1.1))

line1, = ax.plot([], [], lw = 2, color = "r", linestyle = 'dashed')
line2, = ax.plot([], [], lw = 2, color = "b")

def animate(i):
    global x
    global u, uRef
    yRef = uRef[i]
    y    = u[i]
    line1.set_data(x, yRef)
    line2.set_data(x, y)
    return (line1, line2)

anim = animation.FuncAnimation(fig, animate, frames = M + 1, interval = 20)
anim.save('leapfrog.mp4', fps = 30, extra_args=['-vcodec', 'libx264'])
rc('animation', html = 'jshtml') 
anim